In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import geopandas as gpd
import json

# The Data

This data was published by Patrick Ford via Kaggle at https://www.kaggle.com/datasets/patricklford/global-ev-sales-2010-2024.

This goal of this script is to tranform the data into a format that can be visualized in a Tableau dashboard.

In [2]:
# Global data
global_data = pd.read_csv('./Data/Global EV Data 2024.csv')

In [3]:
global_data.head()

,region,category,parameter,mode,powertrain,year,unit,value
0,Australia,Historical,EV stock share,Cars,EV,2011,percent,0.00039
1,Australia,Historical,EV sales share,Cars,EV,2011,percent,0.00650
2,Australia,Historical,EV sales,Cars,BEV,2011,Vehicles,49.00000
3,Australia,Historical,EV stock,Cars,BEV,2011,Vehicles,49.00000
4,Australia,Historical,EV stock,Cars,BEV,2012,Vehicles,220.00000


In [4]:
global_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12654 entries, 0 to 12653
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   region      12654 non-null  object 
 1   category    12654 non-null  object 
 2   parameter   12654 non-null  object 
 3   mode        12654 non-null  object 
 4   powertrain  12654 non-null  object 
 5   year        12654 non-null  int64  
 6   unit        12654 non-null  object 
 7   value       12654 non-null  float64
dtypes: float64(1), int64(1), object(6)
memory usage: 791.0+ KB


In [5]:
global_data.describe()

,year,value
count,12654.000000,1.265400e+04
mean,2019.822112,4.273742e+05
std,5.476494,6.860498e+06
min,2010.000000,1.200000e-06
25%,2016.000000,2.000000e+00
50%,2020.000000,1.300000e+02
75%,2022.000000,5.500000e+03
max,2035.000000,4.400000e+08


# Data Cleaning and Exploration

There are a few things to note about the data after taking a quick glance.
1) There seems to be 6 non-numerical columns. I want to see what values those columns (`category`, `parameter`, `mode`, `powertrain`, and `unit`) can have.
2) Based upon the first few rows of the data, it appears that this data was formatted long where `value` describes the `parameter`. The way I envision the Tableau dashboard to work, having wider data, instead of longer, would work better.
3) Since this data seems to only capture yearly statistics, a year-over-year (YoY) metric may be helpful to visualize.

## Step 1: Values

In [6]:
# Category
global_data['category'].unique()

array(['Historical', 'Projection-STEPS', 'Projection-APS'], dtype=object)

In [7]:
# Parameter
global_data['parameter'].unique()

array(['EV stock share', 'EV sales share', 'EV sales', 'EV stock',
       'EV charging points', 'Electricity demand', 'Oil displacement Mbd',
       'Oil displacement, million lge'], dtype=object)

In [8]:
# Mode
global_data['mode'].unique()

array(['Cars', 'EV', 'Buses', 'Vans', 'Trucks'], dtype=object)

In [9]:
# Powertrain
global_data['powertrain'].unique()

array(['EV', 'BEV', 'PHEV', 'Publicly available fast',
       'Publicly available slow', 'FCEV'], dtype=object)

In [10]:
# Unit
global_data['unit'].unique()

array(['percent', 'Vehicles', 'charging points', 'GWh',
       'Milion barrels per day', 'Oil displacement, million lge'],
      dtype=object)

In [11]:
# Region
global_data['region'].unique()

array(['Australia', 'Austria', 'Belgium', 'Brazil', 'Bulgaria', 'Canada',
       'Chile', 'China', 'Colombia', 'Costa Rica', 'Croatia', 'Cyprus',
       'Czech Republic', 'Denmark', 'Estonia', 'EU27', 'Europe',
       'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Iceland',
       'India', 'Indonesia', 'Ireland', 'Israel', 'Italy', 'Japan',
       'Korea', 'Latvia', 'Lithuania', 'Luxembourg', 'Mexico',
       'Netherlands', 'New Zealand', 'Norway', 'Poland', 'Portugal',
       'Rest of the world', 'Romania', 'Seychelles', 'Slovakia',
       'Slovenia', 'South Africa', 'Spain', 'Sweden', 'Switzerland',
       'Thailand', 'Turkiye', 'United Arab Emirates', 'United Kingdom',
       'USA', 'World'], dtype=object)

Since the focus of this project is historical sales of EVs, I'm going to filter out all other data.

In [12]:
# Filter for relevant data
data_rel = global_data[(global_data['category'] == 'Historical')
                        # & (global_data['region'] != 'World')
                        & (~global_data['parameter'].str.contains('Oil'))
                        & (global_data['powertrain'].str.contains('EV'))
                        & (global_data['year'] < datetime.today().year)
                        ]

In [13]:
data_rel.head()

,region,category,parameter,mode,powertrain,year,unit,value
0,Australia,Historical,EV stock share,Cars,EV,2011,percent,0.00039
1,Australia,Historical,EV sales share,Cars,EV,2011,percent,0.00650
2,Australia,Historical,EV sales,Cars,BEV,2011,Vehicles,49.00000
3,Australia,Historical,EV stock,Cars,BEV,2011,Vehicles,49.00000
4,Australia,Historical,EV stock,Cars,BEV,2012,Vehicles,220.00000


In [14]:
# Pivot data to expand the 'parameter' column
pivoted = pd.pivot_table(data_rel,
                           index = ['year','region','powertrain','mode'],
                           columns = ['parameter'],
                           values = ['value'],
                           aggfunc = 'sum',
                           fill_value = 0)

In [15]:
# Rename the column headers
pivoted.columns = [i[1]
                   for i in pivoted.columns]

In [16]:
# Calculate the year over year changes in metrics
pivoted_yoy = pivoted.reset_index()\
                        .groupby(['region','powertrain','mode'])\
                        .apply(lambda x: x.set_index('year')\
                                           .pct_change())

In [19]:
# Rename the column headers for the YoY data
pivoted_yoy.columns = ['_'.join([i,'yoy'])
                       for i in pivoted_yoy.columns]

In [20]:
# Join the pivoted data (raw numbers) and the YoY data into one dataframe
joined = pd.merge(pivoted,
                  pivoted_yoy,
                  how = 'outer',
                  left_index = True,
                  right_index = True)\
            .reset_index()\
            .sort_values(['region','powertrain','mode','year'])

In [21]:
joined.head(10)

,year,region,powertrain,mode,EV sales,EV sales share,EV stock,EV stock share,Electricity demand,EV sales_yoy,EV sales share_yoy,EV stock_yoy,EV stock share_yoy,Electricity demand_yoy
151,2011,Australia,BEV,Cars,49.0,0.0,49.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
332,2012,Australia,BEV,Cars,170.0,0.0,220.0,0.0,0.0,2.469388,NaN,3.489796,NaN,NaN
533,2013,Australia,BEV,Cars,190.0,0.0,410.0,0.0,0.0,0.117647,NaN,0.863636,NaN,NaN
752,2014,Australia,BEV,Cars,370.0,0.0,780.0,0.0,0.0,0.947368,NaN,0.902439,NaN,NaN
982,2015,Australia,BEV,Cars,760.0,0.0,1500.0,0.0,0.0,1.054054,NaN,0.923077,NaN,NaN
1277,2016,Australia,BEV,Cars,670.0,0.0,2200.0,0.0,0.0,-0.118421,NaN,0.466667,NaN,NaN
1600,2017,Australia,BEV,Cars,1200.0,0.0,3400.0,0.0,0.0,0.791045,NaN,0.545455,NaN,NaN
1945,2018,Australia,BEV,Cars,1800.0,0.0,5200.0,0.0,0.0,0.500000,NaN,0.529412,NaN,NaN
2296,2019,Australia,BEV,Cars,6300.0,0.0,12000.0,0.0,0.0,2.500000,NaN,1.307692,NaN,NaN
2678,2020,Australia,BEV,Cars,5200.0,0.0,17000.0,0.0,0.0,-0.174603,NaN,0.416667,NaN,NaN


# Country Shape File

In order to possibly get a good comprehensive visual on how different countries are performing in Tableau, I would need to merge this data to a .geojson or .shp file to join on. 

In [35]:
with open('C:/Users/Chiga/Desktop/Projects/World_Countries_(Generalized)_9029012925078512962.geojson') as f:
    geojson_data = json.load(f)

In [36]:
country_shp = pd.json_normalize(geojson_data['features'])

In [37]:
country_shp.head()

,type,id,geometry.type,geometry.coordinates,properties.FID,properties.COUNTRY,properties.ISO,properties.COUNTRYAFF,properties.AFF_ISO
0,Feature,1,Polygon,"[[[61.2765541900582, 35.6072450798241], [61.29...",1,Afghanistan,AF,Afghanistan,AF
1,Feature,2,Polygon,"[[[19.57082707295, 41.6852732197367], [19.5819...",2,Albania,AL,Albania,AL
2,Feature,3,Polygon,"[[[4.60335406112642, 36.8879090287794], [4.635...",3,Algeria,DZ,Algeria,DZ
3,Feature,4,Polygon,"[[[-170.743900004405, -14.375554952132], [-170...",4,American Samoa,AS,United States,US
4,Feature,5,Polygon,"[[[1.44583614083058, 42.6019449756544], [1.486...",5,Andorra,AD,Andorra,AD


In [53]:
# check join `region` has associated shapes from country_shp
country_shp_check = pd.merge(joined[['region']],
                             country_shp[['properties.COUNTRY']],
                             how = 'left',
                             left_on = 'region',
                             right_on = 'properties.COUNTRY')

In [54]:
country_shp_check[country_shp_check['properties.COUNTRY'].isna()]['region'].unique()

array(['EU27', 'Europe', 'Rest of the world', 'World'], dtype=object)

In [ ]:
# can keep world
# Remove Europe & Rest of the world
# find match to USA
# korea = south korea?
# EU27 = 27 European Union countries. can remove

## Change USA in `joined['region']`

In [44]:
# What's is the equivalent of USA in country_shp
country_shp[(country_shp['properties.COUNTRY'].str.contains('USA'))
            | (country_shp['properties.COUNTRY'].str.contains('United States'))]

,type,id,geometry.type,geometry.coordinates,properties.FID,properties.COUNTRY,properties.ISO,properties.COUNTRYAFF,properties.AFF_ISO
238,Feature,239,MultiPolygon,"[[[[-76.3950089154603, 39.2299911494413], [-76...",239,United States,US,United States,US
239,Feature,240,MultiPolygon,"[[[[-160.021144980634, -0.398054948442745], [-...",240,United States Minor Outlying Islands,UM,United States,US


In [45]:
# change USA to United States in joined
joined.loc[joined['region'] == 'USA',
            'region'] = 'United States'

## Korea

In [50]:
joined[(joined['region'] == 'Korea')
        & (joined['year'] == 2023)]#['EV sales'].sum()

,year,region,powertrain,mode,EV sales,EV sales share,EV stock,EV stock share,Electricity demand,EV sales_yoy,EV sales share_yoy,EV stock_yoy,EV stock share_yoy,Electricity demand_yoy
4140,2023,Korea,BEV,Buses,2500.0,0.00,4400.0,0.00,0.0,2.906250,NaN,-0.153846,NaN,NaN
4141,2023,Korea,BEV,Cars,120000.0,0.00,460000.0,0.00,0.0,0.000000,NaN,0.533333,NaN,NaN
4142,2023,Korea,BEV,Vans,41000.0,0.00,120000.0,0.00,0.0,0.138889,NaN,0.481481,NaN,NaN
4143,2023,Korea,EV,Buses,0.0,0.97,0.0,0.41,0.0,NaN,2.880000,NaN,-0.267857,NaN
4144,2023,Korea,EV,Cars,0.0,7.90,0.0,2.40,0.0,NaN,-0.070588,NaN,0.411765,NaN
4145,2023,Korea,EV,Vans,0.0,18.00,0.0,4.20,0.0,NaN,0.200000,NaN,0.448276,NaN
4146,2023,Korea,FCEV,Buses,370.0,0.00,650.0,0.00,0.0,1.466667,NaN,1.321429,NaN,NaN
4147,2023,Korea,FCEV,Cars,4300.0,0.00,34000.0,0.00,0.0,-0.570000,NaN,0.172414,NaN,NaN
4148,2023,Korea,PHEV,Cars,12000.0,0.00,59000.0,0.00,0.0,0.090909,NaN,0.035088,NaN,NaN


In [51]:
# how is South Korea represented in shape data
country_shp[(country_shp['properties.COUNTRY'].str.contains('Korea'))]

,type,id,geometry.type,geometry.coordinates,properties.FID,properties.COUNTRY,properties.ISO,properties.COUNTRYAFF,properties.AFF_ISO
164,Feature,165,Polygon,"[[[127.438754212144, 39.4049451061686], [127.4...",165,North Korea,KP,"Korea, Democratic People's Republic of",KP
211,Feature,212,MultiPolygon,"[[[[126.869218306655, 36.0606000956261], [126....",212,South Korea,KR,"Korea, Republic of",KR


In [52]:
# After research, Korea numbers are similar to South Korea numbers
# Change Korea to South Korea
joined.loc[joined['region'] == 'Korea',
            'region'] = 'South Korea'

# Merge `joined` to shapes based on region name
*Note: World won't have an associate shape*

In [64]:
country_shp.columns = [i.split('.')[1] if '.' in i
                       else i
                       for i in country_shp.columns]

In [66]:
country_shp.columns

Index(['type', 'id', 'type', 'coordinates', 'FID', 'COUNTRY', 'ISO',
       'COUNTRYAFF', 'AFF_ISO'],
      dtype='object')

In [70]:
# join country shapes
joined_shp = pd.merge(joined[~joined['region'].isin(['Europe','Rest of the world','EU27'])],
                      country_shp[['COUNTRY','ISO','type','coordinates']],
                      how = 'left',
                      left_on = 'region',
                      right_on = 'COUNTRY')\
                .drop('COUNTRY',
                      axis = 1)

In [71]:
# export joined for data visualization
joined_shp.to_csv('./Data/pivoted_data_yoy_shp.csv',
              index = False)

In [72]:
# export joined without shapes
joined.to_csv('./Data/pivoted_data_yoy.csv',
              index = False)